In [1]:
from data import NumericalField, CategoricalField, Iterator
from data import Dataset
from synthesizer import VGAN_generator, VGAN_discriminator
from synthesizer import LGAN_generator, LGAN_discriminator, LSTM_discriminator
from synthesizer import DCGAN_generator, DCGAN_discriminator
from synthesizer import V_Train
from random import choice

import pandas as pd
import numpy as np
import torch

import json

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import random
import math

In [2]:
VGAN_variable = {
	"batch_size":[64],
	"z_dim":[128],
	"gen_hidden_dim":[200],
	"gen_num_layers":[3],
	"dis_hidden_dim":[200],
	"dis_num_layers":[3],
	"dis_lstm_dim":[200],
	"lr":[0.0002],
	"noise":[0.1]
}

In [3]:
def parameter_search(Model):
	parameters = {}
	if Model == "VGAN":
		variable = VGAN_variable

	# 创建一个存 parameter的dictionary,随机选择一个parameter
	for param in variable.keys():
		parameters[param] = choice(variable[param])
	return parameters

In [35]:
config = { 
        "name": "modified_adult-VGAN-1hot-norm",
		"train": "modified_adult_train.csv",	
		"sample": "modified_adult_train.csv",
		"gmm_cols":[],
		"normalize_cols":[0,2,4,10,11,12],
		"one-hot_cols":[1,3,5,6,7,8,9,13,14],
		"ordinal_cols":[],
		"model": "VGAN",
		"n_epochs":20,
		"steps_per_epoch":10,
		"n_search": 5,
		"rand_search": "yes",
		"train_method":"VTrain"
	}


In [4]:
config = { 
        "name": "modified_adult-VGAN-1hot-norm",
		"train": "modified_adult_train_sample.csv",	
		"sample": "modified_adult_train_sample.csv",
		"gmm_cols":[],
		"normalize_cols":[0,2,4,10,11,12],
		"one-hot_cols":[1,3,5,6,7,8,9,13,14],
		"ordinal_cols":[],
		"model": "VGAN",
		"n_epochs":20,
		"steps_per_epoch":10,
		"n_search": 5,
		"rand_search": "yes",
		"train_method":"VTrain"
	}

In [37]:
# config = { 
#         "name": "modified_adult-VGAN-1hot-norm",
# 		"train": "adult_train.csv",	
# 		"sample": "adult_train.csv",
# 		"gmm_cols":[],
# 		"normalize_cols":[0,2,4,10,11,12],
# 		"one-hot_cols":[1,3,5,6,7,8,9,13,14],
# 		"ordinal_cols":[],
# 		"model": "VGAN",
# 		"n_epochs":10,
# 		"steps_per_epoch":10,
# 		"n_search": 5,
# 		"rand_search": "yes",
# 		"train_method":"VTrain"
# 	}


In [5]:
train = pd.read_csv(config["train"])
n_search = config["n_search"]
search = 1
ratio = 0.9
noise = 0.1
path = "generated/"
fields = []
col_type = []



for i, col in enumerate(list(train)):

    if i in config["normalize_cols"]:
        fields.append((col,NumericalField("normalize")))
        col_type.append("normalize")
    elif i in config["gmm_cols"]:
        fields.append((col, NumericalField("gmm", n=5)))
        col_type.append("gmm")
    elif i in config["one-hot_cols"]:
        fields.append((col, CategoricalField("one-hot", noise=noise)))
        col_type.append("one-hot")
    elif i in config["ordinal_cols"]:
        fields.append((col, CategoricalField("dict")))
        col_type.append("ordinal")
    else:
        fields.append((col, CategoricalField("binary",noise=noise)))
        col_type.append("binary")


# print("train row : {}".format(len(trn)))
# print("sample row: {}".format(len(samp)))




In [6]:
trn, samp = Dataset.split(
    fields = fields,
    path = ".",
    train = config["train"],
    validation = config["sample"],
    format = "csv",
    valid_ratio=ratio
)


In [7]:
fields

[('age', <data.field.NumericalField at 0x237fe10a648>),
 ('workclass', <data.field.CategoricalField at 0x237c3029188>),
 ('fnlwgt', <data.field.NumericalField at 0x237c3029308>),
 ('education', <data.field.CategoricalField at 0x237c30293c8>),
 ('education-num', <data.field.NumericalField at 0x237c3029388>),
 ('marital-status', <data.field.CategoricalField at 0x237c303d408>),
 ('occupation', <data.field.CategoricalField at 0x237c303d248>),
 ('relationship', <data.field.CategoricalField at 0x237c303d208>),
 ('race', <data.field.CategoricalField at 0x237c303d308>),
 ('sex', <data.field.CategoricalField at 0x237c3029948>),
 ('capital-gain', <data.field.NumericalField at 0x237c3029348>),
 ('capital-loss', <data.field.NumericalField at 0x237c303d2c8>),
 ('hours-per-week', <data.field.NumericalField at 0x237c303d748>),
 ('native-country', <data.field.CategoricalField at 0x237c303d788>),
 ('income', <data.field.CategoricalField at 0x237c303d7c8>)]

In [8]:
trn.learn_convert()
samp.learn_convert()

In [9]:
model = config["model"]
train_method = config["train_method"]


param = parameter_search(config["model"])
train_it, sample_it = Iterator.split(
    batch_size = param["batch_size"],
    train = trn,
    validation = samp,
    sort_key = None,
    shuffle = True,
    labels = None,
    square = False,
    pad = None
)
x_dim = train_it.data.shape[1]
col_dim = trn.col_dim
col_ind = trn.col_ind

In [10]:
c_dim = 0
condition = False
sample_times = 1
KL = True
GPU = torch.cuda.is_available()

In [11]:
GPU

True

In [12]:
def compute_kl(real, pred):
    return torch.sum((torch.log(pred + 1e-4) - torch.log(real + 1e-4)) * pred)

def KL_Loss(x_fake, x_real, col_type, col_dim):
    kl = 0.0
    sta = 0
    end = 0
    for i in range(len(col_type)):
        dim = col_dim[i]
        sta = end
        end = sta+dim
        fakex = x_fake[:,sta:end]
        realx = x_real[:,sta:end]
        if col_type[i] == "gmm":
            fake2 = fakex[:,1:]
            real2 = realx[:,1:]
            # column sum
            dist = torch.sum(fake2, dim=0)

            dist = dist / torch.sum(dist)
            real = torch.sum(real2, dim=0)
            real = real / torch.sum(real)
            kl += compute_kl(real, dist)
        else:
            dist = torch.sum(fakex, dim=0)
            dist = dist / torch.sum(dist)
            
            real = torch.sum(realx, dim=0)
            real = real / torch.sum(real)
            
            kl += compute_kl(real, dist)
    return kl


def mean_Loss(x_fake, x_real, col_type, col_dim):
    mean = 0.0
    sta = 0
    end = 0
    for i in range(len(col_type)):
        dim = col_dim[i]
        sta = end
        end = sta+dim
        fakex = x_fake[:,sta:end]
        realx = x_real[:,sta:end]
        if col_type[i] == "gmm":
            fake2 = fakex[:,1:]
            real2 = realx[:,1:]
            dist = torch.mean(fake2, dim=0)
            dist = dist / torch.sum(dist)
            real = torch.mean(real2, dim=0)
            real = real / torch.sum(real)
            mean += torch.sum(abs(real - dist))
        else:
            dist = torch.mean(fakex, dim=0)
            dist = dist / torch.sum(dist)
            
            real = torch.mean(realx, dim=0)
            real = real / torch.sum(real)
            mean += torch.sum(abs(real - dist))
    return mean

In [13]:
def to_df(data,dataset):
    samples = data.reshape(data.shape[0], -1)
    samples = samples[:,:dataset.dim]
    samples = samples.cpu()
    sample_table = dataset.reverse(samples.detach().numpy())
    df = pd.DataFrame(sample_table,columns=dataset.columns)
    return df

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import pandas as pd
fd_data = pd.read_csv("modified_adult_train.csv")
A = fd_data["education-num"]
B = fd_data["capital-gain"]
def build_model():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[1]),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mae',
                optimizer=optimizer)
  return model


FD_model = build_model()

FD_model.fit(
  A, B,epochs=100,batch_size=256)

Epoch 1/100
118/118 [==============================] - 2s 4ms/step - loss: 5049.7759
Epoch 2/100
118/118 [==============================] - 0s 3ms/step - loss: 5019.7246
Epoch 3/100
118/118 [==============================] - 0s 3ms/step - loss: 4960.8486
Epoch 4/100
118/118 [==============================] - 0s 3ms/step - loss: 4864.8960
Epoch 5/100
118/118 [==============================] - 0s 3ms/step - loss: 4723.4243
Epoch 6/100
118/118 [==============================] - 0s 3ms/step - loss: 4528.1572
Epoch 7/100
118/118 [==============================] - 0s 3ms/step - loss: 4270.2485
Epoch 8/100
118/118 [==============================] - 0s 3ms/step - loss: 3942.0671
Epoch 9/100
118/118 [==============================] - 0s 3ms/step - loss: 3534.7566
Epoch 10/100
118/118 [==============================] - 0s 3ms/step - loss: 3039.4968
Epoch 11/100
118/118 [==============================] - 0s 3ms/step - loss: 2448.8123
Epoch 12/100
118/118 [==============================] - 0s 3ms/

In [ ]:
def V_Train_modifed(t, path, sampleloader, G, D, 
epochs, lr, dataloader, z_dim, dataset, 
col_type, sample_times, itertimes = 100, 
steps_per_epoch = None, GPU=False, KL=True):

    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    
    if GPU:
        G.cuda()
        D.cuda()
    G.GPU = True
    
    D_optim = optim.Adam(D.parameters(), lr=lr, weight_decay=0.00001)
    G_optim = optim.Adam(G.parameters(), lr=lr, weight_decay=0.00001)

    # the default # of steps is the # of batches.

    if steps_per_epoch is None:
        steps_per_epoch = len(dataloader)

    for epoch in range(epochs):
        it = 0
        log = open(path+"train_log_"+str(t)+".txt","a+")
        log.write("-----------Epoch {}-----------\n".format(epoch))
        log.close()
        print("-----------Epoch {}-----------\n".format(epoch))
        while it < steps_per_epoch:
            
            # batch 128, x_real.shape = [128,105]
            for x_real in dataloader:
                if GPU:
                    x_real = x_real.cuda()

                ''' train Discriminator '''
                z = torch.randn(x_real.shape[0], z_dim)
                
                if GPU:
                    z = z.cuda()   
                x_fake = G(z)
                
                y_real = D(x_real)
                y_fake = D(x_fake)

                # df_fake = to_df(x_fake,dataset)
                # A = df_fake["education-num"].astype('float32')
                # #predicted = model.predict(A)
                # predicted = FD_model.predict(A)

                #D_fd = sum(abs(predicted-df_fake["capital-gain"]))/y_fake.shape[0]
                D_fd = 0
                #D_fd = sum(abs(predicted.flatten()-df_fake["capital-gain"]))

        


                # 生成 0 和 1，避免  Discriminator 对 Generator 压制
                # D_Loss = -(torch.mean(y_real) - torch.mean(y_fake)) # wgan loss
                fake_label = torch.zeros(y_fake.shape[0], 1)
                real_label = np.ones([y_real.shape[0], 1])
                # Avoid the suppress of Discriminator over Generator
                real_label = real_label * 0.7 + np.random.uniform(0, 0.3, real_label.shape)
                real_label = torch.from_numpy(real_label).float()
                
                if GPU:

                    fake_label = fake_label.cuda()
                    real_label = real_label.cuda()

 
                    
                D_Loss1 = F.binary_cross_entropy(y_real, real_label)
                D_Loss2 = F.binary_cross_entropy(y_fake, fake_label)

                D_Loss = D_Loss1 + D_Loss2 + D_fd
                
                G_optim.zero_grad()
                D_optim.zero_grad()
                D_Loss.backward()
                D_optim.step()

                ''' train Generator '''
                # 生成 一批noisy
                z = torch.randn(x_real.shape[0], z_dim)
                if GPU:
                    z = z.cuda()
                
                x_fake = G(z)
                y_fake = D(x_fake)

                

                # df_fake = to_df(x_fake,dataset)
                # A = df_fake["education-num"].astype('float32')
                # #predicted = model.predict(A)
                # predicted = FD_model.predict(A)
                # generated = df_fake["capital-gain"].astype('float32')

                G_fd = 0
                # G_fd = sum(abs(predicted-generated))/y_fake.shape[0]
                # print("Generator Loss: ",G_fd)
                # log = open(path+"train_log_"+str(t)+".txt","a+")
                # log.write("Generator Loss: \n".format(G_fd))
                # log.close()
                #G_fd = sum(abs(predicted.flatten()-generated))
                
                

                real_label = torch.ones(y_fake.shape[0], 1)
                if GPU:
                    real_label = real_label.cuda()

                G_Loss1 = F.binary_cross_entropy(y_fake, real_label)
                G_Loss2 = mean_Loss(x_fake, x_real, col_type, dataset.col_dim)
                if KL:
                    KL_loss = KL_Loss(x_fake, x_real, col_type, dataset.col_dim)
                    G_Loss = G_Loss1 + KL_loss + G_fd + G_Loss2
                    #G_Loss = G_Loss1 + KL_loss + G_fd
                else:
                    G_Loss = G_Loss1 + G_fd + G_Loss2
                    #G_Loss = G_Loss1 + G_fd

                G_optim.zero_grad()
                D_optim.zero_grad()
                G_Loss.backward()
                G_optim.step()

                it += 1

                if it%itertimes == 0:
                    log = open(path+"train_log_"+str(t)+".txt","a+")
                    log.write("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                    log.close()
                    print("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                if it >= steps_per_epoch:
                    G.eval()

                    for time in range(sample_times):
                        sample_data = None
                        for x_real in sampleloader:
                            z = torch.randn(x_real.shape[0], z_dim)
                            if GPU:
                                z = z.cuda()
                            x_fake = G(z)
                            samples = x_fake
                            samples = samples.reshape(samples.shape[0], -1)
                            samples = samples[:,:dataset.dim]
                            samples = samples.cpu()
                            sample_table = dataset.reverse(samples.detach().numpy())
                            df = pd.DataFrame(sample_table,columns=dataset.columns)
                            if sample_data is None:
                                sample_data = df
                            else:
                                sample_data = sample_data.append(df)
                        sample_data.to_csv(path+'sample_data_{}_{}_{}.csv'.format(t,epoch,time), index = None)
                   # if GPU:
                   #     G.cuda()
                    #    G.GPU = True
                    G.train()
                    break
    return G,D

In [ ]:
gen = VGAN_generator(param["z_dim"], param["gen_hidden_dim"], x_dim, param["gen_num_layers"], col_type, col_ind, condition=condition,c_dim=c_dim)
dis = VGAN_discriminator(x_dim, param["dis_hidden_dim"], param["dis_num_layers"],condition,c_dim)
V_Train_modifed(search, "its_sample/", sample_it, gen, dis, 700, param["lr"], train_it, param["z_dim"], trn, col_type, sample_times,itertimes = 100, steps_per_epoch = config["steps_per_epoch"],GPU=GPU,KL=KL)

In [ ]:
def V_Train_modifed_origin(t, path, sampleloader, G, D, 
epochs, lr, dataloader, z_dim, dataset, 
col_type, sample_times, itertimes = 100, 
steps_per_epoch = None, GPU=False, KL=True):

    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    
    if GPU:
        G.cuda()
        D.cuda()
    G.GPU = True
    
    D_optim = optim.Adam(D.parameters(), lr=lr, weight_decay=0.00001)
    G_optim = optim.Adam(G.parameters(), lr=lr, weight_decay=0.00001)

    # the default # of steps is the # of batches.

    if steps_per_epoch is None:
        steps_per_epoch = len(dataloader)

    for epoch in range(epochs):
        it = 0
        log = open(path+"train_log_"+str(t)+".txt","a+")
        log.write("-----------Epoch {}-----------\n".format(epoch))
        log.close()
        print("-----------Epoch {}-----------\n".format(epoch))
        while it < steps_per_epoch:
            
            # batch 128, x_real.shape = [128,105]
            for x_real in dataloader:
                if GPU:
                    x_real = x_real.cuda()

                ''' train Discriminator '''
                z = torch.randn(x_real.shape[0], z_dim)
                
                if GPU:
                    z = z.cuda()   
                x_fake = G(z)
                
                y_real = D(x_real)
                y_fake = D(x_fake)

                df_fake = to_df(x_fake,dataset)
                A = df_fake["education-num"].astype('float32')
                #predicted = model.predict(A)
                predicted = FD_model.predict(A)
        


                # 生成 0 和 1，避免  Discriminator 对 Generator 压制
                # D_Loss = -(torch.mean(y_real) - torch.mean(y_fake)) # wgan loss
                fake_label = torch.zeros(y_fake.shape[0], 1)
                real_label = np.ones([y_real.shape[0], 1])
                # Avoid the suppress of Discriminator over Generator
                real_label = real_label * 0.7 + np.random.uniform(0, 0.3, real_label.shape)
                real_label = torch.from_numpy(real_label).float()
                
                if GPU:

                    fake_label = fake_label.cuda()
                    real_label = real_label.cuda()

 
                    
                D_Loss1 = F.binary_cross_entropy(y_real, real_label)
                D_Loss2 = F.binary_cross_entropy(y_fake, fake_label)

                D_Loss = D_Loss1 + D_Loss2
                
                G_optim.zero_grad()
                D_optim.zero_grad()
                D_Loss.backward()
                D_optim.step()

                ''' train Generator '''
                # 生成 一批noisy
                z = torch.randn(x_real.shape[0], z_dim)
                if GPU:
                    z = z.cuda()
                
                x_fake = G(z)
                y_fake = D(x_fake)

                                

                real_label = torch.ones(y_fake.shape[0], 1)
                if GPU:
                    real_label = real_label.cuda()

                G_Loss1 = F.binary_cross_entropy(y_fake, real_label)
                G_Loss2 = mean_Loss(x_fake, x_real, col_type, dataset.col_dim)
                if KL:
                    KL_loss = KL_Loss(x_fake, x_real, col_type, dataset.col_dim)
                    #G_Loss = G_Loss1 + KL_loss + G_fd + G_Loss2
                    G_Loss = G_Loss1 + KL_loss
                else:
                    #G_Loss = G_Loss1 + G_fd + G_Loss2
                    G_Loss = G_Loss1

                G_optim.zero_grad()
                D_optim.zero_grad()
                G_Loss.backward()
                G_optim.step()

                it += 1

                if it%itertimes == 0:
                    log = open(path+"train_log_"+str(t)+".txt","a+")
                    log.write("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                    log.close()
                    print("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                if it >= steps_per_epoch:
                    G.eval()

                    for time in range(sample_times):
                        sample_data = None
                        for x_real in sampleloader:
                            z = torch.randn(x_real.shape[0], z_dim)
                            if GPU:
                                z = z.cuda()
                            x_fake = G(z)
                            samples = x_fake
                            samples = samples.reshape(samples.shape[0], -1)
                            samples = samples[:,:dataset.dim]
                            samples = samples.cpu()
                            sample_table = dataset.reverse(samples.detach().numpy())
                            df = pd.DataFrame(sample_table,columns=dataset.columns)
                            if sample_data is None:
                                sample_data = df
                            else:
                                sample_data = sample_data.append(df)
                        sample_data.to_csv(path+'sample_data_{}_{}_{}.csv'.format(t,epoch,time), index = None)
                   # if GPU:
                   #     G.cuda()
                    #    G.GPU = True
                    G.train()
                    break
    return G,D


gen = VGAN_generator(param["z_dim"], param["gen_hidden_dim"], x_dim, param["gen_num_layers"], col_type, col_ind, condition=condition,c_dim=c_dim)
dis = VGAN_discriminator(x_dim, param["dis_hidden_dim"], param["dis_num_layers"],condition,c_dim)
V_Train_modifed_origin(search, "origin_loss_sample/", sample_it, gen, dis, 700, param["lr"], train_it, param["z_dim"], trn, col_type, sample_times,itertimes = 100, steps_per_epoch = config["steps_per_epoch"],GPU=GPU,KL=KL)

In [15]:
def V_Train_modifed_mean(t, path, sampleloader, G, D, 
epochs, lr, dataloader, z_dim, dataset, 
col_type, sample_times, itertimes = 100, 
steps_per_epoch = None, GPU=False, KL=True):

    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    
    if GPU:
        G.cuda()
        D.cuda()
    G.GPU = True
    
    D_optim = optim.Adam(D.parameters(), lr=lr, weight_decay=0.00001)
    G_optim = optim.Adam(G.parameters(), lr=lr, weight_decay=0.00001)

    # the default # of steps is the # of batches.

    if steps_per_epoch is None:
        steps_per_epoch = len(dataloader)

    for epoch in range(epochs):
        it = 0
        log = open(path+"train_log_"+str(t)+".txt","a+")
        log.write("-----------Epoch {}-----------\n".format(epoch))
        log.close()
        print("-----------Epoch {}-----------\n".format(epoch))
        while it < steps_per_epoch:
            
            # batch 128, x_real.shape = [128,105]
            for x_real in dataloader:
                if GPU:
                    x_real = x_real.cuda()

                ''' train Discriminator '''
                z = torch.randn(x_real.shape[0], z_dim)
                
                if GPU:
                    z = z.cuda()   
                x_fake = G(z)
                
                y_real = D(x_real)
                y_fake = D(x_fake)

                df_fake = to_df(x_fake,dataset)
                A = df_fake["education-num"].astype('float32')
                #predicted = model.predict(A)
                predicted = FD_model.predict(A)
        


                # 生成 0 和 1，避免  Discriminator 对 Generator 压制
                # D_Loss = -(torch.mean(y_real) - torch.mean(y_fake)) # wgan loss
                fake_label = torch.zeros(y_fake.shape[0], 1)
                real_label = np.ones([y_real.shape[0], 1])
                # Avoid the suppress of Discriminator over Generator
                real_label = real_label * 0.7 + np.random.uniform(0, 0.3, real_label.shape)
                real_label = torch.from_numpy(real_label).float()
                
                if GPU:

                    fake_label = fake_label.cuda()
                    real_label = real_label.cuda()

 
                    
                D_Loss1 = F.binary_cross_entropy(y_real, real_label)
                D_Loss2 = F.binary_cross_entropy(y_fake, fake_label)

                D_Loss = D_Loss1 + D_Loss2
                
                G_optim.zero_grad()
                D_optim.zero_grad()
                D_Loss.backward()
                D_optim.step()

                ''' train Generator '''
                # 生成 一批noisy
                z = torch.randn(x_real.shape[0], z_dim)
                if GPU:
                    z = z.cuda()
                
                x_fake = G(z)
                y_fake = D(x_fake)

                                

                real_label = torch.ones(y_fake.shape[0], 1)
                if GPU:
                    real_label = real_label.cuda()

                G_Loss1 = F.binary_cross_entropy(y_fake, real_label)
                G_Loss2 = mean_Loss(x_fake, x_real, col_type, dataset.col_dim)
                if KL:
                    KL_loss = KL_Loss(x_fake, x_real, col_type, dataset.col_dim)
                    #G_Loss = G_Loss1 + KL_loss + G_fd + G_Loss2
                    G_Loss = G_Loss1 + KL_loss + G_Loss2
                else:
                    #G_Loss = G_Loss1 + G_fd + G_Loss2
                    G_Loss = G_Loss1 + G_Loss2

                G_optim.zero_grad()
                D_optim.zero_grad()
                G_Loss.backward()
                G_optim.step()

                it += 1

                if it%itertimes == 0:
                    log = open(path+"train_log_"+str(t)+".txt","a+")
                    log.write("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                    log.close()
                    print("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                if it >= steps_per_epoch:
                    G.eval()

                    for time in range(sample_times):
                        sample_data = None
                        for x_real in sampleloader:
                            z = torch.randn(x_real.shape[0], z_dim)
                            if GPU:
                                z = z.cuda()
                            x_fake = G(z)
                            samples = x_fake
                            samples = samples.reshape(samples.shape[0], -1)
                            samples = samples[:,:dataset.dim]
                            samples = samples.cpu()
                            sample_table = dataset.reverse(samples.detach().numpy())
                            df = pd.DataFrame(sample_table,columns=dataset.columns)
                            if sample_data is None:
                                sample_data = df
                            else:
                                sample_data = sample_data.append(df)
                        sample_data.to_csv(path+'sample_data_{}_{}_{}.csv'.format(t,epoch,time), index = None)
                   # if GPU:
                   #     G.cuda()
                    #    G.GPU = True
                    G.train()
                    break
    return G,D


gen = VGAN_generator(param["z_dim"], param["gen_hidden_dim"], x_dim, param["gen_num_layers"], col_type, col_ind, condition=condition,c_dim=c_dim)
dis = VGAN_discriminator(x_dim, param["dis_hidden_dim"], param["dis_num_layers"],condition,c_dim)
V_Train_modifed_mean(search, "mean_loss_sample/", sample_it, gen, dis, 700, param["lr"], train_it, param["z_dim"], trn, col_type, sample_times,itertimes = 100, steps_per_epoch = config["steps_per_epoch"],GPU=GPU,KL=KL)

-----------Epoch 0-----------

-----------Epoch 1-----------

-----------Epoch 2-----------

-----------Epoch 3-----------

-----------Epoch 4-----------

-----------Epoch 5-----------

-----------Epoch 6-----------

-----------Epoch 7-----------

-----------Epoch 8-----------

-----------Epoch 9-----------

-----------Epoch 10-----------

-----------Epoch 11-----------

-----------Epoch 12-----------

-----------Epoch 13-----------

-----------Epoch 14-----------

-----------Epoch 15-----------

-----------Epoch 16-----------

-----------Epoch 17-----------

-----------Epoch 18-----------

-----------Epoch 19-----------

-----------Epoch 20-----------

-----------Epoch 21-----------

-----------Epoch 22-----------

-----------Epoch 23-----------

-----------Epoch 24-----------

-----------Epoch 25-----------

-----------Epoch 26-----------

-----------Epoch 27-----------

-----------Epoch 28-----------

-----------Epoch 29-----------

-----------Epoch 30-----------

-----------Epoch 3

(VGAN_generator(
   (input): Linear(in_features=128, out_features=200, bias=True)
   (inputbn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc0): Linear(in_features=200, out_features=200, bias=True)
   (bn0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc1): Linear(in_features=200, out_features=200, bias=True)
   (bn1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc2): Linear(in_features=200, out_features=200, bias=True)
   (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (output): Linear(in_features=200, out_features=99, bias=True)
   (outputbn): BatchNorm1d(99, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 ),
 VGAN_discriminator(
   (input): Linear(in_features=99, out_features=200, bias=True)
   (Dropout): Dropout(p=0.5, inplace=False)
   (fc0): Linear(in_features=200, out_features=200, bias=True)
   (f

In [17]:
def V_Train_modifed(t, path, sampleloader, G, D, 
epochs, lr, dataloader, z_dim, dataset, 
col_type, sample_times, itertimes = 100, 
steps_per_epoch = None, GPU=False, KL=True):

    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    
    if GPU:
        G.cuda()
        D.cuda()
    G.GPU = True
    
    D_optim = optim.Adam(D.parameters(), lr=lr, weight_decay=0.00001)
    G_optim = optim.Adam(G.parameters(), lr=lr, weight_decay=0.00001)

    # the default # of steps is the # of batches.

    if steps_per_epoch is None:
        steps_per_epoch = len(dataloader)

    for epoch in range(epochs):
        it = 0
        log = open(path+"train_log_"+str(t)+".txt","a+")
        log.write("-----------Epoch {}-----------\n".format(epoch))
        log.close()
        print("-----------Epoch {}-----------\n".format(epoch))
        while it < steps_per_epoch:
            
            # batch 128, x_real.shape = [128,105]
            for x_real in dataloader:
                if GPU:
                    x_real = x_real.cuda()

                ''' train Discriminator '''
                z = torch.randn(x_real.shape[0], z_dim)
                
                if GPU:
                    z = z.cuda()   
                x_fake = G(z)
                
                y_real = D(x_real)
                y_fake = D(x_fake)

                df_fake = to_df(x_fake,dataset)
                A = df_fake["education-num"].astype('float32')
                #predicted = model.predict(A)
                predicted = FD_model.predict(A).flatten()

                D_fd = sum(abs(predicted-df_fake["capital-gain"]))/y_fake.shape[0]
                #D_fd = sum(abs(predicted.flatten()-df_fake["capital-gain"]))

        


                # 生成 0 和 1，避免  Discriminator 对 Generator 压制
                # D_Loss = -(torch.mean(y_real) - torch.mean(y_fake)) # wgan loss
                fake_label = torch.zeros(y_fake.shape[0], 1)
                real_label = np.ones([y_real.shape[0], 1])
                # Avoid the suppress of Discriminator over Generator
                real_label = real_label * 0.7 + np.random.uniform(0, 0.3, real_label.shape)
                real_label = torch.from_numpy(real_label).float()
                
                if GPU:

                    fake_label = fake_label.cuda()
                    real_label = real_label.cuda()

 
                    
                D_Loss1 = F.binary_cross_entropy(y_real, real_label)
                D_Loss2 = F.binary_cross_entropy(y_fake, fake_label)

                D_Loss = D_Loss1 + D_Loss2 + D_fd
                
                G_optim.zero_grad()
                D_optim.zero_grad()
                D_Loss.backward()
                D_optim.step()

                ''' train Generator '''
                # 生成 一批noisy
                z = torch.randn(x_real.shape[0], z_dim)
                if GPU:
                    z = z.cuda()
                
                x_fake = G(z)
                y_fake = D(x_fake)

                

                df_fake = to_df(x_fake,dataset)
                A = df_fake["education-num"].astype('float32')
                #predicted = model.predict(A)
                predicted = FD_model.predict(A).flatten()
                generated = df_fake["capital-gain"].astype('float32')


                G_fd = sum(abs(predicted-generated))/y_fake.shape[0]
                print("Generator Loss: ",G_fd)
                log = open(path+"train_log_"+str(t)+".txt","a+")
                log.write("Generator Loss: \n".format(G_fd))
                log.close()
                #G_fd = sum(abs(predicted.flatten()-generated))
                
                

                real_label = torch.ones(y_fake.shape[0], 1)
                if GPU:
                    real_label = real_label.cuda()

                G_Loss1 = F.binary_cross_entropy(y_fake, real_label)
                G_Loss2 = mean_Loss(x_fake, x_real, col_type, dataset.col_dim)
                if KL:
                    KL_loss = KL_Loss(x_fake, x_real, col_type, dataset.col_dim)
                    G_Loss = G_Loss1 + KL_loss + G_fd
                    #G_Loss = G_Loss1 + KL_loss + G_fd
                else:
                    G_Loss = G_Loss1 + G_fd
                    #G_Loss = G_Loss1 + G_fd

                G_optim.zero_grad()
                D_optim.zero_grad()
                G_Loss.backward()
                G_optim.step()

                it += 1

                if it%itertimes == 0:
                    log = open(path+"train_log_"+str(t)+".txt","a+")
                    log.write("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                    log.close()
                    print("iterator {}, D_Loss:{}, G_Loss:{}\n".format(it,D_Loss.data, G_Loss.data))
                if it >= steps_per_epoch:
                    G.eval()

                    for time in range(sample_times):
                        sample_data = None
                        for x_real in sampleloader:
                            z = torch.randn(x_real.shape[0], z_dim)
                            if GPU:
                                z = z.cuda()
                            x_fake = G(z)
                            samples = x_fake
                            samples = samples.reshape(samples.shape[0], -1)
                            samples = samples[:,:dataset.dim]
                            samples = samples.cpu()
                            sample_table = dataset.reverse(samples.detach().numpy())
                            df = pd.DataFrame(sample_table,columns=dataset.columns)
                            if sample_data is None:
                                sample_data = df
                            else:
                                sample_data = sample_data.append(df)
                        sample_data.to_csv(path+'sample_data_{}_{}_{}.csv'.format(t,epoch,time), index = None)
                   # if GPU:
                   #     G.cuda()
                    #    G.GPU = True
                    G.train()
                    break
    return G,D

gen = VGAN_generator(param["z_dim"], param["gen_hidden_dim"], x_dim, param["gen_num_layers"], col_type, col_ind, condition=condition,c_dim=c_dim)
dis = VGAN_discriminator(x_dim, param["dis_hidden_dim"], param["dis_num_layers"],condition,c_dim)
V_Train_modifed(search, "fd_loss_sample/", sample_it, gen, dis, 700, param["lr"], train_it, param["z_dim"], trn, col_type, sample_times,itertimes = 100, steps_per_epoch = config["steps_per_epoch"],GPU=GPU,KL=KL)

-----------Epoch 0-----------

Generator Loss:  3223.5962591171265
Generator Loss:  2992.1013050079346
Generator Loss:  3159.849536895752
Generator Loss:  3026.00892829895
Generator Loss:  3176.621072769165
Generator Loss:  2503.6302728652954
Generator Loss:  2593.811644554138
Generator Loss:  2597.841583251953
Generator Loss:  2921.8689527511597
Generator Loss:  2831.1655435562134
-----------Epoch 1-----------

Generator Loss:  3142.114610671997
Generator Loss:  3036.4441633224487
Generator Loss:  2776.901756286621
Generator Loss:  3686.7929401397705
Generator Loss:  3267.161720275879
Generator Loss:  3124.369524002075
Generator Loss:  3231.1821823120117
Generator Loss:  2871.365526199341
Generator Loss:  2937.3218717575073
Generator Loss:  3170.7693376541138
-----------Epoch 2-----------

Generator Loss:  3128.6974754333496
Generator Loss:  2995.0192432403564
Generator Loss:  2476.27499294281
Generator Loss:  3101.7833824157715
Generator Loss:  3018.150161743164
Generator Loss:  2848

(VGAN_generator(
   (input): Linear(in_features=128, out_features=200, bias=True)
   (inputbn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc0): Linear(in_features=200, out_features=200, bias=True)
   (bn0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc1): Linear(in_features=200, out_features=200, bias=True)
   (bn1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc2): Linear(in_features=200, out_features=200, bias=True)
   (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (output): Linear(in_features=200, out_features=99, bias=True)
   (outputbn): BatchNorm1d(99, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 ),
 VGAN_discriminator(
   (input): Linear(in_features=99, out_features=200, bias=True)
   (Dropout): Dropout(p=0.5, inplace=False)
   (fc0): Linear(in_features=200, out_features=200, bias=True)
   (f

In [ ]:
def to_df(data,dataset):
    samples = data.reshape(data.shape[0], -1)
    samples = samples[:,:dataset.dim]
    samples = samples.cpu()
    sample_table = dataset.reverse(samples.detach().numpy())
    df = pd.DataFrame(sample_table,columns=dataset.columns)
    return df


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import pandas as pd
fd_data = pd.read_csv("modified_adult_train.csv")
A = fd_data["education-num"]
B = fd_data["capital-gain"]
def build_model():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[1]),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mae',
                optimizer=optimizer)
  return model


FD_model = build_model()

FD_model.fit(
  A, B,epochs=100,batch_size=256)

In [ ]:
import pandas as pd

In [ ]:
fd_data = pd.read_csv("modified_adult_train.csv")

In [ ]:
A_norm=(fd_data["education-num"]-fd_data["education-num"].mean())/fd_data["education-num"].std()
B_norm=(fd_data["capital-gain"]-fd_data["capital-gain"].mean())/fd_data["capital-gain"].std()

In [ ]:
A = fd_data["education-num"]
B = fd_data["capital-gain"]

In [ ]:
def FD_autoencoder():
    data = layers.Input(shape = (None, 1))

    #encoded = layers.Conv1D(1,1,2)(data)
    encoded = layers.Conv1DTranspose(1, 5, strides=2)(data)
    decoded = layers.Conv1D(1, 5, strides=2)(encoded)
    
    #decoded = layers.Conv1DTranspose(1, 5, strides=2)(encoded)


    encoder = keras.Model(data, encoded)
    autoencoder = keras.Model(data, decoded)


    autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError())

    return autoencoder#
autoencoder= FD_autoencoder()
optimizer = tf.keras.optimizers.RMSprop(0.001)
# autoencoder.compile(loss='mse',
#                 optimizer=optimizer,
#                 metrics=['mae', 'mse'])

autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError())
autoencoder.fit(
  A, B,epochs=200,batch_size=256)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[1]),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mae',
                optimizer=optimizer)
  return model

In [ ]:
FD_model = build_model()

In [ ]:
history = FD_model.fit(
  A, B,epochs=100,batch_size=256)

In [ ]:
FD_model.predict(A)